In [1]:
#!pip install deepface
from deepface.commons import functions
from deepface import DeepFace
import tqdm
import os
from neo4j import GraphDatabase, basic_auth

# Retrieve facial database

In [2]:
img_paths = []
for root, dirs, files in os.walk("deepface/tests/dataset/"):
    for file in files:
        if '.jpg' in file:
            img_path = root+file
            img_paths.append(img_path)

# Face recognition model

In [3]:
model = DeepFace.build_model("Facenet")

# Find vector representations

In [4]:
instances = {}
#for img_path in img_paths:
for i in tqdm.tqdm(range(0, len(img_paths))):
    
    img_path = img_paths[i]
    
    #detect and align
    img = functions.preprocess_face(img_path, target_size = (160, 160))
    
    #represent
    embeding = model.predict(img)[0].tolist()
    
    #raw image name without path and extension
    label = img_path.split("/")[-1].split(".")[0]
    
    instances[label] = embeding

100%|██████████| 60/60 [00:11<00:00,  5.24it/s]


# Connect Neo4j

In [5]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "neo4j"))

In [6]:
session = driver.session()

In [22]:
#flush database
result = session.run("MATCH (n) DETACH DELETE n")

# Creating nodes

In [23]:
statements = ""
for img_label in instances.keys():
    statement = "MERGE (%s_face:Face {name:'%s'})" \
                  " SET %s_face.embedding = %s \n" % (img_label, img_label, img_label, instances[img_label])
    statements += statement
    
    """if img_label == 'img1':
        print(statement)"""

In [24]:
with session.begin_transaction() as trx:
    trx.run(statements)
    trx.commit()

# Find distances between nodes

In [25]:
query = """
    MATCH (p1:Face)
    MATCH (p2:Face)
    WHERE p1.name <> p2.name
    RETURN p1.name AS from, p2.name AS to
    , gds.alpha.similarity.euclideanDistance(p1.embedding, p2.embedding) as distance
    ORDER BY distance ASC
    LIMIT 10
"""

results = session.run(query)

In [26]:
for result in results:
    print(result)

<Record from='img51' to='img57' distance=2.43656215816999>
<Record from='img57' to='img51' distance=2.43656215816999>
<Record from='img67' to='img51' distance=2.5293301871888247>
<Record from='img51' to='img67' distance=2.5293301871888247>
<Record from='img67' to='img57' distance=2.6380299746019933>
<Record from='img57' to='img67' distance=2.6380299746019933>
<Record from='img57' to='img5' distance=2.9911645529701962>
<Record from='img5' to='img57' distance=2.9911645529701962>
<Record from='img51' to='img5' distance=3.3633656553453206>
<Record from='img5' to='img51' distance=3.3633656553453206>


# Create edges

In [27]:
create_edge_statement = """
    MATCH (p1:Face)
    MATCH (p2:Face)
    WHERE p1.name <> p2.name
    WITH p1, p2, gds.alpha.similarity.euclideanDistance(p1.embedding, p2.embedding) as distance
    WHERE distance < 10
    MERGE (p1)-[e:distance]-(p2)
    SET e.distance=distance
"""

results = session.run(create_edge_statement)

In [29]:
session.close()
driver.close()